In [8]:
import pandas as pd
import numpy as np
import time
import os
%run ../funcoes_eficiencia.py



In [9]:
input_path  = os.path.join('out','')
output_path = os.path.join('out','')
if not os.path.exists(output_path): 
    os.mkdir(output_path)

In [10]:
primeiro_estagio = {  'qtd_cilindros': 1 ,
                     'cilindros_lubrificados' : 0,
                     'qtd_efeitos' : 2,
                     'diametro_cilindro' :490,
                     'curso_pistao' : 160,
                     'diametro_haste' : 75,
                     'clearance' : 12.5,
                     'rpm' : 710,
                     'vol_deslc_cilindro' : 0,
                     'vazao_volumetrica' : 0,
                     'vel_pistao' : 0}

segundo_estagio = {  'qtd_cilindros': 2 ,
                     'cilindros_lubrificados' : 0,
                     'qtd_efeitos' : 2,
                     'diametro_cilindro' :235,
                     'curso_pistao' : 160,
                     'diametro_haste' : 75,
                     'clearance' : 11.64,
                     'rpm' : 710,
                     'vol_deslc_cilindro' : 0,
                     'vazao_volumetrica' : 0,
                     'vel_pistao' : 0}

In [11]:
eteno = Chemical('ethylene')
etano = Chemical('ethane')
propeno = Chemical('propene')
propano = Chemical('propane')
buteno = Chemical('butene')
butano = Chemical('butane')
hexeno = Chemical('1-hexene')
hexano = Chemical('hexane')

tags = [eteno, etano, propeno, propano, buteno, butano, hexeno, hexano]
   
dados = pd.DataFrame(get_parameters(tags))
#dados

In [12]:
data_segundo_estagio = list(dados_compressor(segundo_estagio).values())
#data_segundo_estagio

In [13]:
df = pd.read_csv(input_path+'df_filtrado.csv.gz', compression = 'gzip', parse_dates = ['Date'], index_col = 'Date')
df.head()

,Eteno,Etano,Propeno,Propano,Buteno,Butano,Hexeno,Hexano,Pres_Desc_2,Pres_Suc_2,Temp_Suc_2,Temp_Desc_2
Date,,,,,,,,,,,,
2015-01-09 20:46:00,37.447,9.021,0.021,30.703,8.978,2.103,3.206,0.435,25.618,5.262,39.994,123.4360
2015-01-09 20:47:00,37.502,9.046,0.021,30.693,8.926,2.064,3.190,0.428,27.166,5.655,40.994,129.9625
2015-01-09 20:48:00,37.503,9.046,0.021,30.693,8.926,2.064,3.190,0.428,26.320,5.543,41.000,133.4770
2015-01-09 20:49:00,37.503,9.046,0.021,30.693,8.926,2.064,3.190,0.428,26.645,5.467,41.000,135.9850
2015-01-09 20:50:00,37.503,9.046,0.021,30.693,8.926,2.064,3.190,0.428,27.356,6.357,41.993,136.4965


In [14]:
#CONVERTENDO AS VARIÁVEIS PARA PASCAL E KELVIN
df['Temp_Suc_2'] = df['Temp_Suc_2'] + 273.15
df['Temp_Desc_2'] = df['Temp_Desc_2'] + 273.15

df['Pres_Suc_2'] = (df['Pres_Suc_2'] + 1.01325)*100000   # BAR
df['Pres_Desc_2'] = (df['Pres_Desc_2'] + 1.01325)*100000   # BAR G 

df.loc[:, 'Eteno':'Hexano'] = df.loc[:, 'Eteno':'Hexano']/100

In [15]:
df.shape

(1588310, 12)

In [16]:
tags_suc = ['Temp_Suc_2','Pres_Suc_2']    
tags_desc = ['Temp_Desc_2', 'Pres_Desc_2']

start_time = time.time()

result = recorrencia(df.head(500), dados, data_segundo_estagio, tags_suc,tags_desc)

time_running = time.time()-start_time
print("--- %s minutos ---" % (time_running/60))

result.head()

--- 0.0371467669804891 minutos ---


,Ef_Politropica,Head_Politropica,Ef_Isentropica,Head_Isentropica,Z_Suc,Z_Desc,MW,T_Suc,r,k,Densidade_Suc,Densidade_Desc
Date,,,,,,,,,,,,
2015-01-09 20:46:00,92.917743,99.280470,92.114467,98.422188,0.945128,0.891820,39.273300,313.144,4.243855,1.179041,10.015188,35.566392
2015-01-09 20:47:00,87.765192,99.991074,86.385142,98.418780,0.942324,0.892909,39.234633,314.144,4.225884,1.179041,10.629620,36.942904
2015-01-09 20:48:00,84.022000,99.924216,82.240683,97.805763,0.943336,0.899690,39.234511,314.150,4.169037,1.179041,10.439650,35.256242
2015-01-09 20:49:00,83.415992,102.000223,81.536523,99.702026,0.944019,0.900928,39.234511,314.150,4.268084,1.179041,10.311161,35.408004
2015-01-09 20:50:00,78.040680,94.456029,75.740620,91.672166,0.936618,0.898855,39.234511,315.143,3.849157,1.179041,11.782723,36.356552


In [12]:
Chemical('methane').eos.V_g

0.02441019502181826

In [13]:
Chemical('methane').eos.V_g_sat(30)

1915632296.8215063

In [14]:
Chemical('methane').eos.V_g

0.02441019502181826

In [15]:
tags_suc = ['Temp_Suc_2','Pres_Suc_2']    
tags_desc = ['Temp_Desc_2', 'Pres_Desc_2']
intervalo = (df.index > '2015-01-01 00:00:00') & (df.index < '2020-01-01 00:00:00')
start_time = time.time()

result = recorrencia(df[intervalo], dados, data_segundo_estagio, tags_suc,tags_desc)

time_running = time.time()-start_time
print("--- %s minutos ---" % (time_running/60))
result.head()



--- 122.6821020325025 minutos ---


,Ef_Politropica,Head_Politropica,Ef_Isentropica,Head_Isentropica,Z_Suc,Z_Desc,MW,T_Suc,r,k,Densidade_Suc,Densidade_Desc
Date,,,,,,,,,,,,
2015-01-06 05:00:00,70.216953,65.470738,67.838724,63.253262,0.950720,0.932259,42.550343,306.150,2.820617,1.179041,9.649738,22.181106
2015-01-06 05:01:00,68.710337,65.784760,66.217385,63.397954,0.950546,0.933369,42.550343,307.145,2.816678,1.179041,9.748193,22.242684
2015-01-06 05:02:00,68.292867,66.192728,65.753966,63.731904,0.950125,0.933077,42.550343,307.150,2.831914,1.179041,9.833025,22.496136
2015-01-06 05:03:00,69.281777,66.724619,66.805655,64.339890,0.950291,0.932515,42.550343,308.144,2.849670,1.179041,9.864256,22.770732
2015-01-06 05:04:00,70.374286,67.650892,67.946654,65.317206,0.949326,0.930053,42.550343,308.150,2.896961,1.179041,10.059527,23.645596


In [16]:
result.to_csv(output_path+'df_eficiencia.csv.gz', compression = 'gzip')